# Error Per Layered Gate (EPLG)

In [3]:
%load_ext autoreload
%autoreload 2

## Set IQM Token if using Resonance.

In [4]:
import os
os.environ["IQM_TOKEN"] = "xxxxxxxxxx"

## Choose (or define) a backend

In [ ]:
from iqm.qiskit_iqm.iqm_provider import IQMProvider

resonance_station="garnet"
iqm_url=f"https://cocos.resonance.meetiqm.com/{resonance_station}"
cal_url=f"https://api.resonance.meetiqm.com/quantum-computers/v1/{resonance_station}/calibrations"
provider = IQMProvider(iqm_url)
backend = provider.get_backend()

## EPLG Configuration

In [ ]:
from iqm.benchmarks.randomized_benchmarking.eplg.eplg import *

In [7]:
EXAMPLE_EPLG = EPLGConfiguration(
    drb_depths=sorted(list(set(np.geomspace(1, 200, num=10, endpoint=True, dtype=int).tolist())), reverse=True),
    drb_circuit_samples=25,
    shots=2**8,
    chain_length=10,#backend.num_qubits - 1,
    chain_path_samples=50,
    num_disjoint_layers=3,
    calibration_url=cal_url,
)

## Run the experiment

In [8]:
benchmark_eplg = EPLGBenchmark(backend, EXAMPLE_EPLG)
run_eplg = benchmark_eplg.run()

2025-03-18 13:06:51,930 - iqm.benchmarks.logging_config - INFO - Generating linear chain path
2025-03-18 13:06:52,129 - iqm.benchmarks.logging_config - INFO - Extracting the path that maximizes total 2Q calibration fidelity
2025-03-18 13:06:59,769 - iqm.benchmarks.logging_config - INFO - Clifford dictionaries imported successfully !
2025-03-18 13:06:59,770 - iqm.benchmarks.logging_config - INFO - Executing parallel Direct RB on qubits [(16, 11), (5, 10), (14, 15), (18, 17), (12, 7), (3, 4)]. Will generate and submit all 25 DRB circuits for each depth [200, 111, 61, 34, 18, 10, 5, 3, 1]
2025-03-18 13:06:59,770 - iqm.benchmarks.logging_config - INFO - Depth 200
2025-03-18 13:08:40,911 - iqm.benchmarks.logging_config - INFO - Submitting batch with 25 circuits corresponding to qubits [16, 11, 5, 10, 14, 15, 18, 17, 12, 7, 3, 4]
C:\Users\PedroFigueroaRomero\anaconda3\envs\iqm-bench\Lib\site-packages\iqm\qiskit_iqm\iqm_provider.py:170: UserWarning: Unknown backend option(s): {'calibration_se

## Perform the analysis

In [20]:
result_eplg = benchmark_eplg.analyze()

2025-03-18 13:37:50,515 - iqm.benchmarks.logging_config - INFO - Post-processing parallel Direct RB on qubits [(16, 11), (5, 10), (14, 15), (18, 17), (12, 7), (3, 4)].
2025-03-18 13:37:50,547 - iqm.benchmarks.logging_config - INFO - Depth 200
2025-03-18 13:37:50,653 - iqm.benchmarks.logging_config - INFO - Depth 111
2025-03-18 13:37:50,761 - iqm.benchmarks.logging_config - INFO - Depth 61
2025-03-18 13:37:50,891 - iqm.benchmarks.logging_config - INFO - Depth 34
2025-03-18 13:37:50,991 - iqm.benchmarks.logging_config - INFO - Depth 18
2025-03-18 13:37:51,066 - iqm.benchmarks.logging_config - INFO - Depth 10
2025-03-18 13:37:51,109 - iqm.benchmarks.logging_config - INFO - Depth 5
2025-03-18 13:37:51,146 - iqm.benchmarks.logging_config - INFO - Depth 3
2025-03-18 13:37:51,177 - iqm.benchmarks.logging_config - INFO - Depth 1
2025-03-18 13:37:51,393 - iqm.benchmarks.logging_config - INFO - Post-processing parallel Direct RB on qubits [(11, 6), (10, 9), (15, 19), (17, 13), (7, 8), (4, 1)].
2

In [21]:
run_eplg.dataset.attrs.keys()

dict_keys(['session_timestamp', 'execution_timestamp', 'backend_configuration_name', 'backend_name', 'benchmark', 'shots', 'max_gates_per_batch', 'calset_id', 'routing_method', 'physical_layout', 'use_dd', 'dd_strategy', 'qubits_array', 'is_eplg', 'parallel_execution', 'depths', 'num_circuit_samples', 'qiskit_optim_level', 'two_qubit_gate_ensembles', 'densities_2q_gates', 'clifford_sqg_probabilities', 'sqg_gate_ensembles', 'parallel_all_0', 0, 'parallel_all_1', 1, 'parallel_all_2', 2, 'backend', 'drb_depths', 'drb_circuit_samples', 'custom_qubits_array', 'chain_length', 'chain_path_samples', 'num_disjoint_layers', 'calibration_url'])

In [25]:
for o in result_eplg.observations:
    if o.name != "EPLG":
        print(f"{o.name} {o.identifier.qubit_indices}: {100*o.value:.2f} +/- {100*o.uncertainty:.2f} %")
    else:
        print(f"\n{o.name} {o.identifier.qubit_indices}: {o.value:.2e} +/- {o.uncertainty:.2e}")

avg_gate_fidelity (16, 11): 98.43 +/- 0.05 %
avg_gate_fidelity (5, 10): 96.60 +/- 0.17 %
avg_gate_fidelity (14, 15): 97.53 +/- 0.04 %
avg_gate_fidelity (18, 17): 98.33 +/- 0.05 %
avg_gate_fidelity (12, 7): 93.15 +/- 0.42 %
avg_gate_fidelity (3, 4): 97.55 +/- 0.09 %
avg_gate_fidelity (11, 6): 97.58 +/- 0.05 %
avg_gate_fidelity (10, 9): 96.84 +/- 0.12 %
avg_gate_fidelity (15, 19): 96.89 +/- 0.10 %
avg_gate_fidelity (17, 13): 97.08 +/- 0.08 %
avg_gate_fidelity (7, 8): 94.86 +/- 0.11 %
avg_gate_fidelity (4, 1): 97.25 +/- 0.10 %
avg_gate_fidelity (6, 5): 97.70 +/- 0.12 %
avg_gate_fidelity (9, 14): 98.08 +/- 0.05 %
avg_gate_fidelity (19, 18): 98.37 +/- 0.04 %
avg_gate_fidelity (13, 12): 97.42 +/- 0.08 %
avg_gate_fidelity (8, 3): 98.05 +/- 0.04 %
avg_gate_fidelity (1, 0): 97.91 +/- 0.05 %
Layer Fidelity (n_qubits=19): 59.89 +/- 1.76 %

EPLG (n_qubits=19): 2.81e-02 +/- 1.76e-02
